# HDD Trajectory 

## Init

In [ ]:
from os import listdir
from os.path import isfile, join
dataset_path='traj3/'
onlyfiles = [f for f in listdir(dataset_path) if isfile(join(dataset_path, f))]

##

import numpy as np
traj_files=[dataset_path+f for f in onlyfiles]
dataframe = [np.loadtxt(traj) for traj in traj_files]
series = [serie.astype('float32') for serie in dataframe];

In [ ]:
from math import *

def calculateRT(xteta,yteta,zteta,xtr,ytr,ztr):

    R1=np.array([[cos(zteta), -sin(zteta), 0 ],
        [sin(zteta),  cos(zteta), 0 ],
        [0,           0,          1 ]])
    R2=np.array([[cos(yteta),  0, sin(yteta) ],
        [0,           1, 0          ],
        [-sin(yteta), 0, cos(yteta) ]] )   
    R3=np.array([[1,           0,           0          ],
        [0,           cos(xteta), -sin(xteta) ],
        [0,           sin(xteta),  cos(xteta) ]])
    R=np.dot(np.dot(R1,R2),R3)
    H=np.zeros((4,4))
    H[0:3,0:3]=R;
    H[0,3]=xtr
    H[1,3]=ytr
    H[2,3]=ztr
    H[3,3]=1
    return H

In [ ]:
concatSeries=np.concatenate((np.array(series),np.array(trajectories)),axis=2)

In [ ]:
import copy

trajectories=list()
for serie in series:
    orijData=copy.copy(serie)
    orijTraj=np.zeros((30,3))
    orijBaseFrame=np.array([[1, 0, 0, 0],
                            [0, 1, 0, 0],
                            [0, 0, 1, 0],
                            [0, 0, 0, 1]])
    for i in range(1,30): 
        H=calculateRT(orijData[i,3],orijData[i,4],orijData[i,5],orijData[i,0],orijData[i,1],orijData[i,2]);
        orijBaseFrame=np.dot(H,orijBaseFrame);
        orijTraj[i,:]=orijBaseFrame[0:3,3]
    trajectories.append(orijTraj)


## LSTM 1

In [ ]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from math import sqrt
import matplotlib
import numpy
import numpy as np
import copy
from numpy import concatenate
from mpl_toolkits.mplot3d import Axes3D
# frame a sequence as a supervised learning problem
lstm_model1=None
scalerSeries=None
scalerTrajectories=None
def timeseries_to_supervised(data, lag=1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis=1)
	return df
 
# scale train and test data to [-1, 1]
def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled
 
# fit an LSTM network to training data
def fit_lstm(train, batch_size, neurons):
    X, y = train[:,0:12], train[:,12:18]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons,batch_input_shape=(batch_size, X.shape[1], X.shape[2])))
    model.add(Dropout(0.2))
    model.add(Dense(6),)
    model.compile(loss='mean_squared_error', optimizer='Adagrad')
    for i in range(1):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
        model.reset_states()
    return model
 
# run a repeated experiment
def experiment(series,trajectories):
    # transform data to be stationary
    global scalerSeries
    global scalerTrajectories
    seriesData=numpy.array(copy.copy(series))    
    scalerSeries= preprocessing.MinMaxScaler(
        feature_range=(-1, 1)).fit(seriesData.reshape(seriesData.shape[0]*seriesData.shape[1],
                                                      seriesData.shape[2]))
    trajectoriesData=numpy.array(copy.copy(trajectories))    
    scalerTrajectories= preprocessing.MinMaxScaler(
        feature_range=(-1, 1)).fit(trajectoriesData.reshape(trajectoriesData.shape[0]*trajectoriesData.shape[1],
                                                            trajectoriesData.shape[2]))

    
    global lstm_model1
    for k in range(15):
        for trajIndex in range(0,700):
            serie=copy.copy(series[trajIndex])
            trajectory=copy.copy(trajectories[trajIndex])
            raw_values = np.concatenate((scalerSeries.transform(copy.copy(serie)),
                                        scalerTrajectories.transform(copy.copy(trajectory))),axis=1)
            # transform data to be supervised learning
            supervised = timeseries_to_supervised(raw_values, 1)
            supervised_values = supervised.values[1:,:]
            supervised_values_x=supervised_values[:,0:12]
            supervised_values_y=supervised_values[:,12:]
            # split data into train and test-sets
            scaled_supervised_values_x= supervised_values_x#scaler.transform(supervised_values_x)
            scaled_supervised_values_y= supervised_values_y#scaler.transform(supervised_values_y)
            dataset=np.concatenate((scaled_supervised_values_x, scaled_supervised_values_y),axis=1)
            # run experiment
            # fit the base model
            if lstm_model1==None:
                lstm_model1 = fit_lstm(dataset, 29, 50)
            else:
                X, y = dataset[:,0:12], dataset[:,12:18]
                X = X.reshape(X.shape[0], 1, X.shape[1])
                for i in range(1):
                    lstm_model1.fit(X, y, epochs=1, batch_size=29, verbose=1, shuffle=False)
                    lstm_model1.reset_states()

# execute the experiment
def run():
	# load dataset
    dataframe = [np.loadtxt(traj) for traj in traj_files]
    #series = [serie.astype('float32') for serie in dataframe];
    # experiment
    results = DataFrame()
    # run experiment
    results['results'] = experiment(series,trajectories)
    # summarize results
    print(results.describe())
    # save results
    results.to_csv('experiment_stateful.csv', index=False)
 
 # entry point
run()

## Visualiazation

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_zlim(0, 0.15)
ax.plot([-0.06,-0.06,0.06,0.06,-0.06],[-0.075,0.075,0.075,-0.075,-0.075],[0,0,0,0,0])
ax.hold(True)
for serie in series:
    orijData=copy.copy(serie)
    orijTraj=np.zeros((29,3))
#    print orijData
    orijBaseFrame=np.array([[1, 0, 0, 0],
                   [0, 1, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])
    for i in range(1,28): 
        H=calculateRT(orijData[i,3],orijData[i,4],orijData[i,5],orijData[i,0],orijData[i,1],orijData[i,2]);
        orijBaseFrame=np.dot(H,orijBaseFrame);
        orijTraj[i+1,:]=orijBaseFrame[0:3,3]
    ax.plot(orijTraj[:,0],orijTraj[:,1],orijTraj[:,2])
plt.show()

## LSTM 1 Predictions

In [ ]:
diff_plot_list=list()
orij_plot_list=list()
test_plot_list=list()
orj_Translation_list=list()
test_Translation_list=list()
diff_Translation_list=list()
def experiment(series,trajectories):
    # transform data to be stationary
    global lstm1_model1
    global scalerSeries
    global scalerTrajectories
    for trajIndex in range(700,800):
        serie=copy.copy(series[trajIndex])
        trajectory=copy.copy(trajectories[trajIndex])
        
        raw_values = np.concatenate((scalerSeries.transform(copy.copy(serie)),
                                    scalerTrajectories.transform(copy.copy(trajectory))),axis=1)

        
        Xhat=raw_values[:-1,0:12]
        X = np.zeros((29, 1, 12))
        Xk=Xhat[0,:]
        for i in range(29):
            X[i,:]=Xk.reshape(1, 1, 12)
        testBaseFrame=[[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]]
        for i in range(28):
            
            #X = Xhat.reshape(13, 1, 10)
            #print X
            yhat = lstm_model1.predict(X, batch_size=29)
            Xk=yhat[i,:]
            X[i+1,0,0:6]=Xk.reshape(1, 1, 6)
            temp =copy.copy(X[i+1,0,0:9])
            tempinv=scalerSeries.inverse_transform(temp.reshape(1,9))
            H=calculateRT(tempinv[0,3],tempinv[0,4],tempinv[0,5],tempinv[0,0],tempinv[0,1],tempinv[0,2]);
            testBaseFrame=np.dot(H,testBaseFrame);
            tempPos=testBaseFrame[0:3,3]
            tempPosScaled=scalerTrajectories.transform(tempPos.reshape(1,3))
            X[i+1,0,9:12]=tempPosScaled.reshape(1,1,3)
            lstm_model1.reset_states()            
        testData =scalerSeries.inverse_transform(X[:,0,:9])
        testTraj=np.zeros((30,3))
        orijTraj=np.zeros((30,3))
        testBaseFrame=[[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]]
        orijBaseFrame=[[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]]
        for i in range(29): 
            H=calculateRT(testData[i,3],testData[i,4],testData[i,5],testData[i,0],testData[i,1],testData[i,2]);
            testBaseFrame=np.dot(H,testBaseFrame);
            testTraj[i+1,:]=testBaseFrame[0:3,3];
            H=calculateRT(orijData[i,3],orijData[i,4],orijData[i,5],orijData[i,0],orijData[i,1],orijData[i,2]);
            orijBaseFrame=np.dot(H,orijBaseFrame);
            orijTraj[i+1,:]=orijBaseFrame[0:3,3];
            
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.set_zlim(0, 0.15)
        ax.plot([-0.06,-0.06,0.06,0.06,-0.06],[-0.075,0.075,0.075,-0.075,-0.075],[0,0,0,0,0])
        ax.hold(True)
        ax.scatter(-serie[0,6],serie[0,7])
        diffTraj=orijTraj-testTraj
        diff_plot_list.append(np.sqrt(diffTraj[:,0]**2+diffTraj[:,1]**2+diffTraj[:,2]**2))
        orij_plot_list.append(np.sqrt(orijTraj[:,0]**2+orijTraj[:,1]**2+orijTraj[:,2]**2))
        test_plot_list.append(np.sqrt(testTraj[:,0]**2+testTraj[:,1]**2+testTraj[:,2]**2))
        diffData=orijData[1:,:]-testData
        orj_Translation_list.append(np.sqrt(orijData[:,0]**2+orijData[:,1]**2+orijData[:,2]**2))
        test_Translation_list.append(np.sqrt(testData[:,0]**2+testData[:,1]**2+testData[:,2]**2))
        diff_Translation_list.append(np.sqrt(diffData[:,0]**2+diffData[:,1]**2+diffData[:,2]**2))

        ax.plot(orijTraj[:,0],orijTraj[:,1],orijTraj[:,2],color='green')
        ax.plot(testTraj[:,0],testTraj[:,1],testTraj[:,2],color='red')
        plt.show()
        lstm_model1.reset_states()

dataframe = [np.loadtxt(traj) for traj in traj_files]
series = [serie.astype('float32') for serie in dataframe];
# experiment
results = DataFrame()
# run experiment
results['results'] = experiment(series,trajectories)

## Error Plots

In [ ]:
asd=np.mean(np.array(diff_plot_list),axis=0)
asdvar=np.std(np.array(diff_plot_list),axis=0)
dsa=np.mean(np.array(orij_plot_list),axis=0)
dsavar=np.std(np.array(orij_plot_list),axis=0)
fsa=np.mean(np.array(test_plot_list),axis=0)
fsavar=np.std(np.array(test_plot_list),axis=0)
#asdasd=np.mean(np.abs(np.array(orij_plot_list)-np.array(test_plot_list)),axis=0)
aver_error=np.mean(np.array(diff_plot_list))
aver_error
x=np.array([i for i in range(0,30)])
plt.plot(asd,color='red')
plt.hold(True)
plt.fill_between(x,asd+asdvar,asd-asdvar,alpha=0.3,color='pink')
plt.hold(True)
plt.plot(dsa,color='green')
plt.hold(True)
plt.fill_between(x,dsa+dsavar,dsa-dsavar,alpha=0.3,color='lightgreen')
plt.hold(True)
plt.plot(fsa,color='blue')
plt.hold(True)
plt.fill_between(x,fsa+fsavar,fsa-fsavar,alpha=0.3,color='lightblue')
plt.hold(True)
axes = plt.gca()
axes.set_ylim([0,0.10])
axes.set_xlim([0,28])
#plt.plot(asdasd,color='black')
plt.show()

In [ ]:
asd=np.mean(np.array(diff_plot_list20),axis=0)
asdvar=np.std(np.array(diff_plot_list20),axis=0)
dsa=np.mean(np.array(orij_plot_list20),axis=0)
dsavar=np.std(np.array(orij_plot_list20),axis=0)
fsa=np.mean(np.array(test_plot_list20),axis=0)
fsavar=np.std(np.array(test_plot_list20),axis=0)
#asdasd=np.mean(np.abs(np.array(orij_plot_list)-np.array(test_plot_list)),axis=0)
aver_error=np.mean(np.array(diff_plot_list1))
aver_error
x=np.array([i for i in range(0,30)])
plt.plot(asd,color='red')
plt.hold(True)
plt.fill_between(x,asd+asdvar,asd-asdvar,alpha=0.3,color='pink')
plt.hold(True)
plt.plot(dsa[:-1],color='green')
plt.hold(True)
plt.fill_between(x[:-1],dsa[:-1]+dsavar[:-1],dsa[:-1]-dsavar[:-1],alpha=0.3,color='lightgreen')
plt.hold(True)
plt.plot(fsa,color='blue')
plt.hold(True)
plt.fill_between(x,fsa+fsavar,fsa-fsavar,alpha=0.3,color='lightblue')
plt.hold(True)
axes = plt.gca()
axes.set_ylim([0,0.10])
axes.set_xlim([0,28])

#plt.plot(asdasd,color='black')
plt.show()

In [ ]:
asd=np.mean(np.array(diff_Translation_list),axis=0)
asdvar=np.std(np.array(diff_Translation_list),axis=0)
dsa=np.mean(np.array(orj_Translation_list),axis=0)
dsavar=np.std(np.array(orj_Translation_list),axis=0)

fsa=np.mean(np.array(test_Translation_list),axis=0)
fsavar=np.std(np.array(test_Translation_list),axis=0)

asdasd=np.mean(np.abs(np.array(orj_Translation_list)[:,:-1]-np.array(test_Translation_list)),axis=0)
x=np.array([i for i in range(0,29)])

aver_error=np.mean(np.array(diff_plot_list))
aver_error
plt.plot(x,asd,color='red')
plt.hold(True)
plt.fill_between(x,asd+asdvar,asd-asdvar,alpha=0.3,color='pink')
plt.hold(True)
plt.plot(dsa[:-1],color='green')
plt.hold(True)
plt.fill_between(x,dsa[:-1]+dsavar[:-1],dsa[:-1]-dsavar[:-1],alpha=0.3,color='lightgreen')
plt.hold(True)
plt.plot(fsa,color='blue')
plt.hold(True)
plt.fill_between(x,fsa+fsavar,fsa-fsavar,alpha=0.3,color='lightblue')
plt.hold(True)
#plt.plot(fsa-dsa[:-1],color='black')
#plt.plot(asdasd,color='black')

plt.show()

In [ ]:
diff_plot_list=list()
orij_plot_list=list()
test_plot_list=list()
orj_Translation_list=list()
test_Translation_list=list()
diff_Translation_list=list()
def experiment(series):
    # transform data to be stationary
    count=0;
    #
    global lstm1_model1
    global scaler
    for serie in series[700:1000]:
        orijData=serie
        scaled_serie=scaler.transform(serie)
        Xhat=scaled_serie[:-1,0:9]

        X = np.zeros((29, 1, 9))
        Xk=Xhat[0:15,:]
        X[0:15,:]=Xk.reshape(15, 1, 9)
        for i in range(15,29):
            X[i,0,:]=Xk[-1,:].reshape(1, 1, 9)
        
        for i in range(15,28):
            
            #X = Xhat.reshape(13, 1, 10)
            #print X
            yhat = lstm_model1.predict(X, batch_size=29)
            Xk=yhat[i,:]
            X[i+1,0,0:6]=Xk.reshape(1, 1, 6)
            lstm_model1.reset_states()
        testData =scaler.inverse_transform(X.reshape(29,9))
        testTraj=np.zeros((30,3))
        orijTraj=np.zeros((30,3))
        testBaseFrame=[[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]]
        orijBaseFrame=[[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]]
        for i in range(29): 
            H=calculateRT(testData[i,3],testData[i,4],testData[i,5],testData[i,0],testData[i,1],testData[i,2]);
            testBaseFrame=np.dot(H,testBaseFrame);
            testTraj[i+1,:]=testBaseFrame[0:3,3];
            H=calculateRT(orijData[i,3],orijData[i,4],orijData[i,5],orijData[i,0],orijData[i,1],orijData[i,2]);
            orijBaseFrame=np.dot(H,orijBaseFrame);
            orijTraj[i+1,:]=orijBaseFrame[0:3,3];
            
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.set_zlim(0, 0.15)
        ax.plot([-0.06,-0.06,0.06,0.06,-0.06],[-0.075,0.075,0.075,-0.075,-0.075],[0,0,0,0,0])
        ax.hold(True)
        ax.scatter(-serie[0,6],serie[0,7])
        diffTraj=orijTraj-testTraj
        diff_plot_list.append(np.sqrt(diffTraj[:,0]**2+diffTraj[:,1]**2+diffTraj[:,2]**2))
        orij_plot_list.append(np.sqrt(orijTraj[:,0]**2+orijTraj[:,1]**2+orijTraj[:,2]**2))
        test_plot_list.append(np.sqrt(testTraj[:,0]**2+testTraj[:,1]**2+testTraj[:,2]**2))
        diffData=orijData[:-1,:]-testData
        orj_Translation_list.append(np.sqrt(orijData[:,0]**2+orijData[:,1]**2+orijData[:,2]**2))
        test_Translation_list.append(np.sqrt(testData[:,0]**2+testData[:,1]**2+testData[:,2]**2))
        diff_Translation_list.append(np.sqrt(diffData[:,0]**2+diffData[:,1]**2+diffData[:,2]**2))

        ax.plot(orijTraj[:,0],orijTraj[:,1],orijTraj[:,2],color='green')
        ax.plot(testTraj[:,0],testTraj[:,1],testTraj[:,2],color='red')     
        lstm_model1.reset_states()
        plt.show()

dataframe = [np.loadtxt(traj) for traj in traj_files]
series = [serie.astype('float32') for serie in dataframe];
# experiment
results = DataFrame()
# run experiment
results['results'] = experiment(series)

In [ ]:
totalCount=0
for plot in diff_plot_list:
    countzero=0
    finalPos=False
    for var in plot:
        if var<0.01:
            countzero=countzero+1
    if plot[29]<0.01:
        finalPos=True
        totalCount=totalCount+1
    countzero
    finalPos
print totalCount
    


In [ ]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
import matplotlib
import numpy
import numpy as np
from numpy import concatenate
from mpl_toolkits.mplot3d import Axes3D
# frame a sequence as a supervised learning problem
lstm_model2=None
def fit_lstm(train, batch_size, neurons):
    X, y = train[:, 0:6], train[:,6:9]
    X = X.reshape(X.shape[0], 1, X.shape[1])
    model = Sequential()
    model.add(LSTM(neurons,batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
    model.add(Dropout(0.2))
    model.add(Dense(3))
    model.compile(loss='mean_squared_error', optimizer='Adagrad')
    for i in range(5):
        model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
        model.reset_states()
    return model

def experiment(series):
    # transform data to be stationary
    count=0;
    global lstm_model2
    for serie in series:
        count=count+1
        raw_values = scaler.transform(serie)
        # transform data to be supervised learning
        raw_values = raw_values[:-1,:]
        scaled_supervised_values_x= raw_values[:,0:6]#scaler.transform(supervised_values_x)
        scaled_supervised_values_y= raw_values[:,6:9]#scaler.transform(supervised_values_y)
        dataset=np.concatenate((scaled_supervised_values_x, scaled_supervised_values_y),axis=1)
        # run experiment
        # fit the base model
        if count<700:
            print '.'
            if lstm_model2==None:
                lstm_model2 = fit_lstm(dataset, 29, 32)
            else:
                X, y = dataset[:, 0:6], dataset[:,6:9]
                X = X.reshape(X.shape[0], 1, X.shape[1])
                for i in range(2):
                    lstm_model2.fit(X, y, epochs=1, batch_size=29, verbose=1, shuffle=False)
                    lstm_model2.reset_states()

def run():
	# load dataset
    dataframe = [np.loadtxt(traj) for traj in traj_files]
    series = [serie.astype('float32') for serie in dataframe];
    # experiment
    results = DataFrame()
    # run experiment
    results['results'] = experiment(series)
    # summarize results
    print(results.describe())
    # save results
    results.to_csv('experiment_stateful.csv', index=False)
 
 # entry point
run()

In [ ]:
pointPredictionList=list()
testData=0
def experiment(series):
    # transform data to be stationary
    count=0;
    #
    global lstm_model2
    global scaler 
    global testData
    global serie
    for serie in series[700:1000]:
        scaled_serie=scaler.transform(serie)
        scaled_serie=scaled_serie[:-1,:]
        X = scaled_serie.reshape(29, 1, 9)
        X = X[:,:,0:6]
        y = lstm_model2.predict(X, batch_size=29)
        lstm_model2.reset_states()
        testData=np.concatenate((X.reshape(29,6),y),1)
        testData =scaler.inverse_transform(testData)        #Xhat=yhat
        fig = plt.figure()
        ax = fig.add_subplot(111, projection='3d')
        ax.plot([-0.06,-0.06,0.06,0.06,-0.06],[-0.075,0.075,0.075,-0.075,-0.075],[0,0,0,0,0])
        ax.hold(True)
        ax.scatter(-serie[0,6],serie[0,7],0)
        #ax.plot(serie[:,0],serie[:,1],serie[:,2])
        diff=serie[:-1,6:8]-testData[:,6:8]
        pointPredictionList.append(np.sqrt(diff[:,0]**2+diff[:,1]**2))
        ax.scatter(-testData[25:,6],testData[25:,7],0,c='#FF0000')
        #print np.concatenate((-testData[:,6],testData[:,7]))-np.matlib.repmat([-serie[0,6],serie[0,7]],1,29)
        plt.show()
        lstm_model2.reset_states()
        
dataframe = [np.loadtxt(traj) for traj in traj_files]
series = [serie.astype('float32') for serie in dataframe];
# experiment
results = DataFrame()
# run experiment
results['results'] = experiment(series)

In [ ]:
asd=np.mean(np.array(pointPredictionList),axis=0)
asdvar=np.std(np.array(pointPredictionList),axis=0)
plt.plot(x[:-1],asd,color='red')
plt.hold(True)
plt.fill_between(x[:-1],asd+asdvar,asd-asdvar,alpha=0.3,color='pink')
plt.hold(True)
plt.show()

In [ ]:
asd.shape

In [ ]:
diff_plot_list1=list()
orij_plot_list1=list()
test_plot_list1=list()
diff_plot_list5=list()
orij_plot_list5=list()
test_plot_list5=list()
diff_plot_list10=list()
orij_plot_list10=list()
test_plot_list10=list()
diff_plot_list15=list()
orij_plot_list15=list()
test_plot_list15=list()
diff_plot_list20=list()
orij_plot_list20=list()
test_plot_list20=list()
diff_plot_list25=list()
orij_plot_list25=list()
test_plot_list25=list()
def experiment(series):
    # transform data to be stationary
    count=0;
    #
    global lstm_model
    global lstm_model2
    global scaler 
    for serie in series[700:800]:
        orijData=serie
        scaled_serie=scaler.transform(serie)
        scaled_serie=scaled_serie[:-1,:]
        X = scaled_serie.reshape(29, 1, 9)
        X = X[:,:,0:6]
        yks=np.zeros((29,1,3));        
        testPlots=list();
        for i in range(1,29):
            Xcur=np.zeros((29,1,6))
            Xcur[:i,:,:]=X[:i,:,:]
            for j in range(i,29):
                Xcur[j,:,:]=Xcur[i-1,:,:]
            y = lstm_model2.predict(Xcur, batch_size=29)
            lstm_model2.reset_states()
            yks[i-1,:,:]=y[i-1,:].reshape((1,1,3))
            X2cur=np.zeros((29,1,9))   
            X2cur[:,:,:6]=Xcur
            for j in range(0,29):
                X2cur[j,:,6:]=yks[i-1,:,:];
            for j in range(i,29):
                yhat = lstm_model1.predict(X2cur, batch_size=29)
                Xhat=yhat
                Xk=Xhat[i-1,:]
                X2cur[i,0,0:6]=Xk.reshape(1, 1, 6)
                lstm_model1.reset_states()            
            testPlots.append(X2cur)
        #fig = plt.figure()
        #ax = fig.add_subplot(111, projection='3d')
        #ax.plot([-0.06,-0.06,0.06,0.06,-0.06],[-0.075,0.075,0.075,-0.075,-0.075],[0,0,0,0,0])
        #ax.hold(True)
        #ax.scatter(-serie[0,6],serie[0,7],0,color='blue')
        
        orijTraj=np.zeros((30,3))
        orijBaseFrame=[[1, 0, 0, 0],
                       [0, 1, 0, 0],
                       [0, 0, 1, 0],
                       [0, 0, 0, 1]]
        for i in range(29): 
            H=calculateRT(orijData[i,3],orijData[i,4],orijData[i,5],orijData[i,0],orijData[i,1],orijData[i,2]);
            orijBaseFrame=np.dot(H,orijBaseFrame);
            orijTraj[i+1,:]=orijBaseFrame[0:3,3];        
        
        #ax.plot(orijTraj[:,0],orijTraj[:,1],orijTraj[:,2],color='blue')
        predictedNextPos=np.zeros((29,3));
        for i in range(0,28):
            testData =scaler.inverse_transform(testPlots[i].reshape((29,9)))
            testBaseFrame=[[1, 0, 0, 0],
                           [0, 1, 0, 0],
                           [0, 0, 1, 0],
                           [0, 0, 0, 1]]
            testTraj=np.zeros((30,3))
            for j in range(29): 
                H=calculateRT(testData[j,3],testData[j,4],testData[j,5],testData[j,0],testData[j,1],testData[j,2]);
                testBaseFrame=np.dot(H,testBaseFrame);
                testTraj[j+1,:]=testBaseFrame[0:3,3];
            #ax.plot(testTraj[:,0],testTraj[:,1],testTraj[:,2],color='red'); 
            predictedNextPos[i+1,:]=testTraj[i,0:3]
            #ax.scatter(serie[i+1,0],serie[i+1,1],serie[i+1,2],c='#0000FF')
            #ax.scatter(testData[i,0],testData[i,1],testData[i,2],c='#00FF00')
            diffTraj=orijTraj[:,:3]-testTraj[:,:3]
            if(i==0):
                diff_plot_list1.append(np.sqrt(diffTraj[:,0]**2+diffTraj[:,1]**2+diffTraj[:,2]**2))
                orij_plot_list1.append(np.sqrt(orijTraj[:,0]**2+orijTraj[:,1]**2+orijTraj[:,2]**2))
                test_plot_list1.append(np.sqrt(testTraj[:,0]**2+testTraj[:,1]**2+testTraj[:,2]**2))
            elif(i==4):
                diff_plot_list5.append(np.sqrt(diffTraj[:,0]**2+diffTraj[:,1]**2+diffTraj[:,2]**2))
                orij_plot_list5.append(np.sqrt(orijTraj[:,0]**2+orijTraj[:,1]**2+orijTraj[:,2]**2))
                test_plot_list5.append(np.sqrt(testTraj[:,0]**2+testTraj[:,1]**2+testTraj[:,2]**2))
            elif(i==9):
                diff_plot_list10.append(np.sqrt(diffTraj[:,0]**2+diffTraj[:,1]**2+diffTraj[:,2]**2))
                orij_plot_list10.append(np.sqrt(orijTraj[:,0]**2+orijTraj[:,1]**2+orijTraj[:,2]**2))
                test_plot_list10.append(np.sqrt(testTraj[:,0]**2+testTraj[:,1]**2+testTraj[:,2]**2))
            elif(i==14):
                diff_plot_list15.append(np.sqrt(diffTraj[:,0]**2+diffTraj[:,1]**2+diffTraj[:,2]**2))
                orij_plot_list15.append(np.sqrt(orijTraj[:,0]**2+orijTraj[:,1]**2+orijTraj[:,2]**2))
                test_plot_list15.append(np.sqrt(testTraj[:,0]**2+testTraj[:,1]**2+testTraj[:,2]**2))
            elif(i==19):
                diff_plot_list20.append(np.sqrt(diffTraj[:,0]**2+diffTraj[:,1]**2+diffTraj[:,2]**2))
                orij_plot_list20.append(np.sqrt(orijTraj[:,0]**2+orijTraj[:,1]**2+orijTraj[:,2]**2))
                test_plot_list20.append(np.sqrt(testTraj[:,0]**2+testTraj[:,1]**2+testTraj[:,2]**2))
        #ax.plot(predictedNextPos[:,0],predictedNextPos[:,1],predictedNextPos[:,2],color='green')
        #plt.show()
        
        
dataframe = [np.loadtxt(traj) for traj in traj_files]
series = [serie.astype('float32') for serie in dataframe];
# experiment
results = DataFrame()
# run experiment
results['results'] = experiment(series)